In [3]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as FS
from pyspark.sql import window as wn


In [5]:
spark = SparkSession.builder.master("local[2]").appName("Partition").getOrCreate()

In [46]:
users = [
    (1,"Taukir",'taukir@gmail.com',"8010339935"),
    (2,"Khan",'khan@gmail.com',"8010359935"),
    (3,"Rahul",'rahul@gmail.com',"9810339935"),
    (4,"Chohan",'chohan@gmail.com',"8985339935"),
    (5,"Rohit",'Rohit@gmail.com',"9325339935"),
    (6,"Sab",'sab@gmail.com',"5120339935"),
    (7,"sonu",'jim@gmail.com',"8010009935"),
    (8,"Lavan",'lavan@gmail.com',"8090139935"),
    (9,"Reddy",'reddy@gmail.com',"8010332135"),
    (10,"Kate",'kate@gmail.com',"+110339900"),
    
]
user_columns = ["user_id",'name','email','phone']

address = [
    (1,"Meerut",'Uttar Pradesh','India'),
    (1,"Ghaziabad",'Uttar Pradesh','India'),
     (2,"Kanpur",'Uttar Pradesh','India'),
     (3,"Ghazibad",'Uttar Pradesh','India'),
    (4,"Meerut",'Uttar Pradesh','India'),
    (5,"Panipath",'Hayana','India'),
     (6,"Sonipath",'Hayana','India'),
     (6,"Hisar",'Hayana','India'),
     (8,"Mumbai",'Maharashtra','India'),
     (9,"Pune",'Maharashtra','India'),
    (10,"Toronto",'Onterio','Canada')
    ]
address_columns = ["user_id",'city','state','country']

In [47]:
user_df = spark.createDataFrame(data = users,schema = user_columns)
user_df.show(5)

+-------+------+----------------+----------+
|user_id|  name|           email|     phone|
+-------+------+----------------+----------+
|      1|Taukir|taukir@gmail.com|8010339935|
|      2|  Khan|  khan@gmail.com|8010359935|
|      3| Rahul| rahul@gmail.com|9810339935|
|      4|Chohan|chohan@gmail.com|8985339935|
|      5| Rohit| Rohit@gmail.com|9325339935|
+-------+------+----------------+----------+
only showing top 5 rows



In [53]:
address_df = spark.createDataFrame(data = address,schema = address_columns)
address_df.show(5)

+-------+---------+-------------+-------+
|user_id|     city|        state|country|
+-------+---------+-------------+-------+
|      1|   Meerut|Uttar Pradesh|  India|
|      1|Ghaziabad|Uttar Pradesh|  India|
|      2|   Kanpur|Uttar Pradesh|  India|
|      3| Ghazibad|Uttar Pradesh|  India|
|      4|   Meerut|Uttar Pradesh|  India|
+-------+---------+-------------+-------+
only showing top 5 rows



In [59]:
join_df = user_df.join(address_df,user_df.user_id == address_df.user_id).\
select(user_df.name,user_df.email,user_df.phone,address_df.city,address_df.state,address_df.country)
join_df.show()

+------+----------------+----------+---------+-------------+-------+
|  name|           email|     phone|     city|        state|country|
+------+----------------+----------+---------+-------------+-------+
|Taukir|taukir@gmail.com|8010339935|   Meerut|Uttar Pradesh|  India|
|Taukir|taukir@gmail.com|8010339935|Ghaziabad|Uttar Pradesh|  India|
|  Khan|  khan@gmail.com|8010359935|   Kanpur|Uttar Pradesh|  India|
| Rahul| rahul@gmail.com|9810339935| Ghazibad|Uttar Pradesh|  India|
|Chohan|chohan@gmail.com|8985339935|   Meerut|Uttar Pradesh|  India|
| Rohit| Rohit@gmail.com|9325339935| Panipath|       Hayana|  India|
|   Sab|   sab@gmail.com|5120339935| Sonipath|       Hayana|  India|
|   Sab|   sab@gmail.com|5120339935|    Hisar|       Hayana|  India|
| Lavan| lavan@gmail.com|8090139935|   Mumbai|  Maharashtra|  India|
| Reddy| reddy@gmail.com|8010332135|     Pune|  Maharashtra|  India|
|  Kate|  kate@gmail.com|+110339900|  Toronto|      Onterio| Canada|
+------+----------------+---------

In [79]:
join_df.rdd.getNumPartitions()

1

In [96]:
# d1 = join_df.repartition(numPartitions = 3)
join_df.repartition("state").explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Exchange hashpartitioning(state#685, 200), REPARTITION_BY_COL, [plan_id=4578]
   +- Project [name#602, email#603, phone#604, city#684, state#685, country#686]
      +- SortMergeJoin [user_id#601L], [user_id#683L], Inner
         :- Sort [user_id#601L ASC NULLS FIRST], false, 0
         :  +- Exchange hashpartitioning(user_id#601L, 200), ENSURE_REQUIREMENTS, [plan_id=4572]
         :     +- Filter isnotnull(user_id#601L)
         :        +- Scan ExistingRDD[user_id#601L,name#602,email#603,phone#604]
         +- Sort [user_id#683L ASC NULLS FIRST], false, 0
            +- Exchange hashpartitioning(user_id#683L, 200), ENSURE_REQUIREMENTS, [plan_id=4573]
               +- Filter isnotnull(user_id#683L)
                  +- Scan ExistingRDD[user_id#683L,city#684,state#685,country#686]




In [93]:
d1.rdd.getNumPartitions()

1

In [87]:
d1.show()

+------+----------------+----------+---------+-------------+-------+
|  name|           email|     phone|     city|        state|country|
+------+----------------+----------+---------+-------------+-------+
|Taukir|taukir@gmail.com|8010339935|Ghaziabad|Uttar Pradesh|  India|
|  Kate|  kate@gmail.com|+110339900|  Toronto|      Onterio| Canada|
| Rahul| rahul@gmail.com|9810339935| Ghazibad|Uttar Pradesh|  India|
|Chohan|chohan@gmail.com|8985339935|   Meerut|Uttar Pradesh|  India|
|   Sab|   sab@gmail.com|5120339935|    Hisar|       Hayana|  India|
|Taukir|taukir@gmail.com|8010339935|   Meerut|Uttar Pradesh|  India|
| Rohit| Rohit@gmail.com|9325339935| Panipath|       Hayana|  India|
| Lavan| lavan@gmail.com|8090139935|   Mumbai|  Maharashtra|  India|
| Reddy| reddy@gmail.com|8010332135|     Pune|  Maharashtra|  India|
|   Sab|   sab@gmail.com|5120339935| Sonipath|       Hayana|  India|
|  Khan|  khan@gmail.com|8010359935|   Kanpur|Uttar Pradesh|  India|
+------+----------------+---------

In [88]:
d1.write.option("header",True).mode("overwrite").csv(base_path + "/tmp")

In [8]:
base_path = "../../data/hudsonu-product/"
customer = spark.read.csv(base_path + "Customer.csv",header = True,sep = ',')
customer.show(5)

+-----------+----------+-----------+--------------------+--------------+-----+-----+--------------------+------------+
|customer_id|first_name|  last_name|             address|          city|state|  zip|               email|       phone|
+-----------+----------+-----------+--------------------+--------------+-----+-----+--------------------+------------+
|      10005|    Martie|     Feasby| 8915 Mitchell Place|Grand Junction|   CO|81505|mfeasbyaf@printfr...|970-222-2301|
|      10006|   Tiertza|   Swainson|   689 Drewry Avenue|    Saint Paul|   MN|55166|tswainsonqz@gmail...|651-259-9540|
|      10012|      Nana|   Di Lucia|3 Village Green C...|    Boca Raton|   FL|33499|ndiluciadc@statco...|561-923-6152|
|      10014| Pierrette|    Hassent|    081 Mendota Pass|       Madison|   WI|53710|phassent3u@netsca...|608-241-2936|
|      10015|    Britta|MacSkeaghan|    8 Di Loreto Park|       Buffalo|   NY|14210|bmacskeaghanae@de...|716-947-1700|
+-----------+----------+-----------+------------

In [27]:
customer = customer.select('city').groupBy('city').count()

customer.count()

customer.write.option("header",True).partitionBy("city").mode("overwrite") .csv(base_path + "/tmp/zipcodes-state")

In [23]:
# base_path = "../../data/hudsonu-product/"
# Netsale_Line = spark.read.csv(base_path + "Netsale_Line.csv",header = True,sep = ',')
# Netsale_Line.show(5)

In [28]:
customer.count()

457